# a notebook to make a quick and dirty retina centered video from standard Pupil Player export video

It'll make one retina centered video from a standard Pupil Payer export video

and also a video of the gaze trace

I'll put them together in Adobe Premier or, via this script. We'll see lol

In [1]:
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from scipy import signal
from scipy.signal import savgol_filter


In [2]:
%matplotlib 

Using matplotlib backend: TkAgg


Getcher paths straight

In [3]:
#freemocap session
session_id = 'sesh_2022-05-07_17_15_05_pupil_wobble_juggle_0'
freemocap_data_folder = Path('C:/Users/jonma/Dropbox/FreeMoCapProject/FreeMocap_Data')
session_folder_path = freemocap_data_folder / session_id
pupil_data_path = session_folder_path / 'pupil_000'
pupil_data_exports_path = pupil_data_path / 'exports' / '000'
pupil_export_video_path = pupil_data_exports_path / 'world.mp4'
pupil_world_video_path = pupil_data_path / 'world.mp4'

# #argp pilot_0
# pupil_data_path = Path(r'H:\Other computers\My Computer_MocapComputer\Wirth_ARGP\ARGP_Main\data\2022_04_29\pupil_data')
# pupil_data_exports_path = pupil_data_path / 'exports' / 'Wirth_Pilot_ARGP_2022-04-09_export'
# pupil_export_video_path = pupil_data_exports_path / 'argp_pilot_May2022.mp4'


pupil_world_video_path = pupil_data_path / 'world.mp4'


gaze_positions_path =  pupil_data_exports_path / 'gaze_positions.csv'



load stuff 

In [4]:
#open cv video capture object
pupil_export_video_cap_object = cv2.VideoCapture(str(pupil_export_video_path))

#pupil labs - gaze data
pupil_dataframe = pd.read_csv(gaze_positions_path)

get some info from the video cap

In [5]:
video_width = int(pupil_export_video_cap_object.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height =  int(pupil_export_video_cap_object.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_framerate = int(pupil_export_video_cap_object.get(cv2.CAP_PROP_FPS))
print(f'video_width : {video_width }')
print(f'video_height: {video_height}')
print(f'video_framerate: {video_framerate}')


video_width : 1280
video_height: 720
video_framerate: 59


gaze locations

In [6]:
norm_pos_x  = pupil_dataframe.norm_pos_x
norm_pos_y = pupil_dataframe.norm_pos_y



In [7]:
# fig = plt.Figure(123)
# fig.plot(norm_pos_x, label = 'norm_pos_x')
# fig.plot(norm_pos_y, label ='norm_pos_x')

In [8]:

gaze_on_screen_x = norm_pos_x * video_width
gaze_on_screen_y = norm_pos_y * video_height

world_camera_frame_index_all = pupil_dataframe.world_index

zero-lag 4th order butterworth filter with a 7Hz cutoff

In [9]:


order = 5
cutoff=10**-1
b, a = signal.butter(order, cutoff)
gaze_on_screen_x_filtered = signal.filtfilt(b, a, gaze_on_screen_x)
gaze_on_screen_y_filtered = signal.filtfilt(b, a, gaze_on_screen_y)

print(f"nans in gaze_on_screen_x: {np.isnan(gaze_on_screen_x).sum()}")
print(f"nans in gaze_on_screen_y: {np.isnan(gaze_on_screen_y).sum()}")
print(f"nans in gaze_on_screen_x_filtered: {np.isnan(gaze_on_screen_x_filtered).sum()}")
print(f"nans in gaze_on_screen_y_filtered: {np.isnan(gaze_on_screen_y_filtered).sum()}")


nans in gaze_on_screen_x: 0
nans in gaze_on_screen_y: 0
nans in gaze_on_screen_x_filtered: 0
nans in gaze_on_screen_y_filtered: 0


In [10]:
gaze_on_screen_x_filtered

array([604.55319855, 604.96959393, 605.31206249, ..., 475.75659332,
       475.6162116 , 475.47394065])

In [11]:

plt.cla()
fig2 = plt.Figure()
ax = plt.gca()
ax.plot(gaze_on_screen_x, label = 'gaze_on_screen_x_raw')
ax.plot(gaze_on_screen_y, label = 'gaze_on_screen_y_raw')
ax.plot(gaze_on_screen_x_filtered, label = 'gaze_on_screen_x_filtered')
ax.plot(gaze_on_screen_y_filtered, label = 'gaze_on_screen_y_filtered')
ax.set_ylim((0-100, video_width+100))
ax.legend()


squash data down so each frame is the average of all data recorded on that frame

In [12]:
number_of_frames = np.max(world_camera_frame_index_all)
number_of_frames

gaze_on_screen_x_downsampled = []
gaze_on_screen_y_downsampled = []

for this_world_frame_index in range(number_of_frames):
    this_frame_data = world_camera_frame_index_all == this_world_frame_index
    this_frame_x_data = gaze_on_screen_x_filtered[this_frame_data]
    this_frame_y_data = gaze_on_screen_y_filtered[this_frame_data]
    gaze_on_screen_x_downsampled.append(np.nanmedian(this_frame_x_data))
    gaze_on_screen_y_downsampled.append(np.nanmedian(this_frame_y_data))

c:\Users\jonma\miniconda3\envs\freemocap-env\lib\site-packages\numpy\lib\nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [ ]:

print(f"nans in gaze_on_screen_x: {np.isnan(gaze_on_screen_x).sum()}")
print(f"nans in gaze_on_screen_y: {np.isnan(gaze_on_screen_y).sum()}")
print(f"nans in gaze_on_screen_x_filtered: {np.isnan(gaze_on_screen_x_filtered).sum()}")
print(f"nans in gaze_on_screen_y_filtered: {np.isnan(gaze_on_screen_y_filtered).sum()}")
print(f"nans in gaze_on_screen_x_downsampled: {np.isnan(gaze_on_screen_x_downsampled).sum()}")
print(f"nans in gaze_on_screen_y_downsampled: {np.isnan(gaze_on_screen_y_downsampled).sum()}")


In [ ]:

# order = 4
# cutoff=10**-1
# b, a = signal.butter(order, cutoff)
# gaze_on_screen_x_filtered = signal.filtfilt(b, a, gaze_on_screen_x_downsampled)
# gaze_on_screen_y_filtered = signal.filtfilt(b, a, gaze_on_screen_y_downsampled)

print(f"nans in gaze_on_screen_x: {np.isnan(gaze_on_screen_x).sum()}")
print(f"nans in gaze_on_screen_y: {np.isnan(gaze_on_screen_y).sum()}")
print(f"nans in gaze_on_screen_x_filtered: {np.isnan(gaze_on_screen_x_filtered).sum()}")
print(f"nans in gaze_on_screen_y_filtered: {np.isnan(gaze_on_screen_y_filtered).sum()}")

# plt.close('all')
# fig2 = plt.Figure()
# ax = plt.gca()
# # ax.plot(gaze_on_screen_x_downsampled, label = 'gaze_on_screen_x_downsampled')
# # ax.plot(gaze_on_screen_y_downsampled, label = 'gaze_on_screen_y_downsampled')
# ax.plot(gaze_on_screen_x_filtered, label = 'gaze_on_screen_x_filtered')
# ax.plot(gaze_on_screen_y_filtered, label = 'gaze_on_screen_y_filtered')
# ax.set_ylim((0-100, video_width+100))
# ax.legend()


In [ ]:

#smoooooooooth, just a bit
smoothWinLength = 5
smoothOrder = 3

gaze_on_screen_x_downsampled_smoothed = savgol_filter(gaze_on_screen_x_downsampled, smoothWinLength, smoothOrder)
gaze_on_screen_y_downsampled_smoothed = savgol_filter(gaze_on_screen_y_downsampled, smoothWinLength, smoothOrder)

In [ ]:
plt.close("all")
fig2 = plt.Figure()
ax = plt.gca()
ax.plot(gaze_on_screen_x_downsampled, '.-', label = 'gaze_on_screen_x_downsampled')
ax.plot(gaze_on_screen_y_downsampled, '.-',label = 'gaze_on_screen_y_downsampled')
ax.plot(gaze_on_screen_x_downsampled_smoothed, '.-', label = 'gaze_on_screen_x_downsampled_smoothed')
ax.plot(gaze_on_screen_y_downsampled_smoothed, '.-',label = 'gaze_on_screen_y_downsampled_smoothed')

ax.set_ylim((0-100, video_width+100))
ax.legend(loc='upper left')



In [ ]:
print(f"nans in gaze_on_screen_x_downsampled: {np.isnan(gaze_on_screen_x_downsampled).sum()}")
print(f"nans in gaze_on_screen_y_downsampled: {np.isnan(gaze_on_screen_y_downsampled).sum()}")

gaze_x_df = pd.DataFrame(gaze_on_screen_x_downsampled)
gaze_x_df.interpolate(method = 'linear', inplace = True)
gaze_on_screen_x_downsampled_gapfilled =  gaze_x_df.to_numpy()

gaze_y_df = pd.DataFrame(gaze_on_screen_y_downsampled)
gaze_y_df.interpolate(method = 'linear', inplace = True)
gaze_on_screen_y_downsampled_gapfilled =  gaze_y_df.to_numpy()

print(f"nans in gaze_on_screen_x_downsampled_gapfilled: {np.isnan(gaze_on_screen_x_downsampled_gapfilled).sum()}")
print(f"nans in gaze_on_screen_y_downsampled_gapfilled: {np.isnan(gaze_on_screen_y_downsampled_gapfilled).sum()}")

In [ ]:



big_blank_image = np.ones((int(video_height)*2, int(video_width)*2, 3), dtype=np.uint8)



In [ ]:
# pupil_world_video_cap_object = cv2.VideoCapture(str(pupil_world_video_path))

# frame_num = 0
# success = True
# while success:    
#     success, image_raw = pupil_world_video_cap_object.read()
#     if not success:
#         break
#     frame_num += 1
# print(f"frame_num: {frame_num}")

create circular mask

from =- https://stackoverflow.com/questions/61516526/how-to-use-opencv-to-crop-circular-image

In [ ]:
#define circles
radius = video_height #pixels
x_center = video_width
y_center = video_height

# draw filled circles in white on black background as masks
circular_mask = np.zeros_like(big_blank_image)
circular_mask = cv2.circle(circular_mask, (x_center,y_center), radius, (255,255,255), -1)



In [ ]:
video_framerate

In [ ]:
pupil_world_video_cap_object = cv2.VideoCapture(str(pupil_world_video_path))

this_retinal_aligned_image = big_blank_image.copy()
this_retinal_aligned_image[:, video_width-video_height:(video_width-video_height)+2*video_height,:]

retinal_video_save_path = pupil_data_exports_path / 'retinal_aligned_video.mp4'
retinal_video_writer = cv2.VideoWriter(str(retinal_video_save_path),
                                       cv2.VideoWriter_fourcc(*"MP4V"),
                                       video_framerate, 
                                       (this_retinal_aligned_image.shape[0], this_retinal_aligned_image.shape[0]))

success = True
for frame_number in range(gaze_on_screen_x_downsampled_gapfilled.shape[0]):
    success, image= pupil_world_video_cap_object.read()
    if not success:
        print('failed to read frame')
        break

    # image  = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    this_retinal_aligned_image = big_blank_image.copy()

    #pull out gaze data to align image to retinal coordinates
    gx = int(gaze_on_screen_x_downsampled_gapfilled[frame_number])
    gy = int(gaze_on_screen_y_downsampled_gapfilled[frame_number])

    bottom_edge_row = int(video_height)*2-(int(video_height)-gy)
    top_edge_row = int(video_height) -(int(video_height)-gy)
    left_col = (int(video_width)-gx)
    right_col = int(video_width)+(int(video_width)-gx)

    cols =  np.arange(top_edge_row,bottom_edge_row)

    #if things are going off screen, use whatever gaze data worked most recently
    if top_edge_row < 0:
        gy = previous_gy
        top_edge_row = int(video_height) -(int(video_height)-gy)
        bottom_edge_row = int(video_height)*2-(int(video_height)-gy)

    if bottom_edge_row > int(video_height)*2:
        gy = previous_gy
        top_edge_row = int(video_height) -(int(video_height)-gy)
        bottom_edge_row = int(video_height)*2-(int(video_height)-gy)
    
    if left_col < 0:
        gx = previous_gx
        right_col = int(video_width)+(int(video_width)-gx)
        left_col = (int(video_width)-gx)
    
    if right_col > int(video_width)*2:
        gx = previous_gx
        right_col = int(video_width)+(int(video_width)-gx)
        left_col = (int(video_width)-gx)

    previous_gx = gx    
    previous_gy = gy
    
    this_retinal_aligned_image[top_edge_row:bottom_edge_row, left_col:right_col,:] = image

    this_retinal_aligned_image[circular_mask==0] = 0


    this_retinal_aligned_image = this_retinal_aligned_image[:, video_width-video_height:(video_width-video_height)+2*video_height,:]

    this_retinal_aligned_image = cv2.line(this_retinal_aligned_image, 
                                            (int(video_height), 0), 
                                            (int(video_height), int(video_height)*2), 
                                            (0,0,0,255), 
                                            5)
    this_retinal_aligned_image = cv2.line(this_retinal_aligned_image, 
                                            (0, int(video_height)), 
                                            (int(video_width)*2, 
                                            int(video_height)), 
                                            (0,0,0,255), 
                                            5)
    this_retinal_aligned_image = cv2.line(this_retinal_aligned_image, 
                                            (int(video_height), 0), 
                                            (int(video_height), int(video_height)*2), 
                                            (255,255,255,255), 
                                            1,)
    this_retinal_aligned_image = cv2.line(this_retinal_aligned_image, 
                                            (0, int(video_height)), 
                                            (int(video_width)*2, 
                                            int(video_height)), 
                                            (255,255,255,255), 
                                            1)
    this_retinal_aligned_image = cv2.circle(this_retinal_aligned_image, 
                                            (int(video_height), int(video_height)),
                                            20,
                                            (0,0,0,255), 
                                            5)
    this_retinal_aligned_image = cv2.circle(this_retinal_aligned_image, 
                                            (int(video_height), int(video_height)),
                                            20,
                                            (255,255,255,255), 
                                            1)

    # plt.cla()
    # plt.imshow(this_retinal_aligned_image[:,:,:3]) 
    # plt.show()
    # plt.pause(0.1)

    retinal_video_writer.write(this_retinal_aligned_image) 

    if frame_number%100 == 0:
        print(f'frame_number: {frame_number}')

retinal_video_writer.release()